# Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src import SMOTE
from src import CFS
from src import metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

# Start matlab service

In [2]:
eng = engi.start_matlab()
eng.addpath(r'src/matlab_CTKCCA/',nargout=0)
eng.addpath(r'src/matlab_KS/',nargout=0)

# variables

In [3]:
result_path = 'result/result.csv'
repeats = 20
ratio = 0.1
lrank = 70
reg = 1E-5

# Data loading and Normalizing Data

In [8]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

#     print(commit_guru_df.columns)
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
#     df = understand_df
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
#     s_df,s_cols = apply_cfs(df)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    df = pd.concat([X,y],axis = 1)
    return df

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

# Matlab integration

## Matlab integration - KS

In [6]:
def KS(source_df,target_df):
    mat_source_df = mat.double(source_df.values.T.tolist())
    mat_target_df = mat.double(target_df.values.T.tolist())
    X = eng.HDP_KS(mat_source_df,mat_target_df,nargout=4)
    train_X,train_y = np.array(X[0]),np.array(X[1]).tolist()[0]
    test_X,test_y = np.array(X[2]),np.array(X[3]).tolist()[0]
    return train_X,train_y,test_X,test_y

# Teting using original Data

## get train test data

In [ ]:

precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()
for s_project in projects:
    try:
        if s_project not in precision_list.keys():
            precision_list[s_project] = {}
            recall_list[s_project] = {}
            pf_list[s_project] = {}
            f1_list[s_project] = {}
            g_list[s_project] = {}
            auc_list[s_project] = {} 
        source_df = load_data(s_project)
        source_df = apply_smote(source_df)
        for d_project in projects:
            try:
                target_df = load_data(d_project)
                # Transforming metrics
                trasformed_train_X,trasformed_train_y,trasformed_test_X,trasformed_test_y = KS(source_df,target_df)
        #         train_df = pd.DataFrame(trasformed_train_X)
        #         train_df['Bugs'] = trasformed_train_y
        #         train_df = apply_smote(train_df)
        #         trasformed_train_y = train_df.Bugs
        #         trasformed_train_X = train_df.drop('Bugs',axis = 1)
                #Training Model & Predicting
                t_clf = LogisticRegression()
                t_clf.fit(trasformed_train_X,trasformed_train_y)
                t_predicted = t_clf.predict(trasformed_test_X)
                # Calculating metrics
                abcd = metrices.measures(trasformed_test_y,t_predicted)
                pf = abcd.get_pf()
                recall = abcd.calculate_recall()
                precision = abcd.calculate_precision()
                f1 = abcd.calculate_f1_score()
                g_score = abcd.get_g_score()
                auc = roc_auc_score(trasformed_test_y, t_predicted)
                # Storing Performance scores
                precision_list[s_project][d_project] = precision
                recall_list[s_project][d_project] = recall
                pf_list[s_project][d_project] = pf
                f1_list[s_project][d_project] = f1
                g_list[s_project][d_project] = g_score
                auc_list[s_project][d_project] = auc
                print(classification_report(trasformed_test_y, t_predicted))
            except:
                continue
    except:
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
with open('results/Performance/KS_100.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

              precision    recall  f1-score   support

         0.0       0.90      0.65      0.76       462
         1.0       0.50      0.83      0.62       189

    accuracy                           0.71       651
   macro avg       0.70      0.74      0.69       651
weighted avg       0.79      0.71      0.72       651

              precision    recall  f1-score   support

         0.0       0.52      0.61      0.56       838
         1.0       0.52      0.44      0.48       826

    accuracy                           0.52      1664
   macro avg       0.52      0.52      0.52      1664
weighted avg       0.52      0.52      0.52      1664

              precision    recall  f1-score   support

         0.0       0.45      0.02      0.05     12345
         1.0       0.29      0.93      0.44      5330

    accuracy                           0.30     17675
   macro avg       0.37      0.48      0.25     17675
weighted avg       0.40      0.30      0.17     17675

              preci

              precision    recall  f1-score   support

         0.0       0.46      0.54      0.50        24
         1.0       0.63      0.56      0.59        34

    accuracy                           0.55        58
   macro avg       0.55      0.55      0.55        58
weighted avg       0.56      0.55      0.55        58

              precision    recall  f1-score   support

         0.0       0.86      0.66      0.75       568
         1.0       0.58      0.81      0.68       335

    accuracy                           0.72       903
   macro avg       0.72      0.74      0.71       903
weighted avg       0.76      0.72      0.72       903

              precision    recall  f1-score   support

         0.0       0.90      0.67      0.77        98
         1.0       0.38      0.74      0.51        27

    accuracy                           0.69       125
   macro avg       0.64      0.71      0.64       125
weighted avg       0.79      0.69      0.71       125

              preci

              precision    recall  f1-score   support

         0.0       0.38      0.09      0.15       253
         1.0       0.38      0.78      0.51       178

    accuracy                           0.38       431
   macro avg       0.38      0.44      0.33       431
weighted avg       0.38      0.38      0.30       431

              precision    recall  f1-score   support

         0.0       0.74      0.70      0.72       437
         1.0       0.40      0.44      0.42       198

    accuracy                           0.62       635
   macro avg       0.57      0.57      0.57       635
weighted avg       0.63      0.62      0.63       635

              precision    recall  f1-score   support

         0.0       0.61      0.76      0.67       205
         1.0       0.62      0.44      0.52       180

    accuracy                           0.61       385
   macro avg       0.61      0.60      0.60       385
weighted avg       0.61      0.61      0.60       385

              preci

              precision    recall  f1-score   support

         0.0       0.61      0.66      0.63      1355
         1.0       0.46      0.41      0.43       960

    accuracy                           0.56      2315
   macro avg       0.54      0.53      0.53      2315
weighted avg       0.55      0.56      0.55      2315

              precision    recall  f1-score   support

         0.0       0.81      0.08      0.15      1685
         1.0       0.28      0.95      0.43       629

    accuracy                           0.32      2314
   macro avg       0.55      0.52      0.29      2314
weighted avg       0.67      0.32      0.23      2314

              precision    recall  f1-score   support

         0.0       0.55      0.03      0.06     14163
         1.0       0.27      0.93      0.41      5333

    accuracy                           0.28     19496
   macro avg       0.41      0.48      0.24     19496
weighted avg       0.47      0.28      0.16     19496

              preci

              precision    recall  f1-score   support

         0.0       0.54      0.08      0.15      3225
         1.0       0.43      0.90      0.58      2447

    accuracy                           0.44      5672
   macro avg       0.48      0.49      0.36      5672
weighted avg       0.49      0.44      0.33      5672

              precision    recall  f1-score   support

         0.0       0.44      0.69      0.54      2147
         1.0       0.72      0.48      0.57      3543

    accuracy                           0.56      5690
   macro avg       0.58      0.58      0.56      5690
weighted avg       0.61      0.56      0.56      5690

              precision    recall  f1-score   support

         0.0       0.41      0.67      0.51      1315
         1.0       0.69      0.44      0.54      2241

    accuracy                           0.52      3556
   macro avg       0.55      0.55      0.52      3556
weighted avg       0.59      0.52      0.53      3556

              preci

...

              precision    recall  f1-score   support

         0.0       0.58      0.08      0.14      2325
         1.0       0.48      0.94      0.63      2085

    accuracy                           0.48      4410
   macro avg       0.53      0.51      0.39      4410
weighted avg       0.53      0.48      0.37      4410

              precision    recall  f1-score   support

         0.0       0.57      0.95      0.72      3587
         1.0       0.48      0.06      0.10      2707

    accuracy                           0.57      6294
   macro avg       0.53      0.51      0.41      6294
weighted avg       0.53      0.57      0.45      6294

              precision    recall  f1-score   support

         0.0       0.51      0.74      0.60      1793
         1.0       0.68      0.44      0.53      2274

    accuracy                           0.57      4067
   macro avg       0.59      0.59      0.57      4067
weighted avg       0.60      0.57      0.56      4067

              preci

              precision    recall  f1-score   support

         0.0       0.86      0.76      0.81      2697
         1.0       0.32      0.47      0.38       653

    accuracy                           0.70      3350
   macro avg       0.59      0.61      0.59      3350
weighted avg       0.75      0.70      0.72      3350

              precision    recall  f1-score   support

         0.0       0.85      0.97      0.91      6152
         1.0       0.19      0.04      0.07      1081

    accuracy                           0.83      7233
   macro avg       0.52      0.50      0.49      7233
weighted avg       0.75      0.83      0.78      7233

              precision    recall  f1-score   support

         0.0       0.64      0.55      0.59      3173
         1.0       0.42      0.53      0.47      2015

    accuracy                           0.54      5188
   macro avg       0.53      0.54      0.53      5188
weighted avg       0.56      0.54      0.54      5188

              preci

              precision    recall  f1-score   support

         0.0       0.82      0.59      0.69       225
         1.0       0.30      0.57      0.39        68

    accuracy                           0.59       293
   macro avg       0.56      0.58      0.54       293
weighted avg       0.70      0.59      0.62       293

              precision    recall  f1-score   support

         0.0       0.88      0.66      0.75        32
         1.0       0.48      0.77      0.59        13

    accuracy                           0.69        45
   macro avg       0.68      0.71      0.67        45
weighted avg       0.76      0.69      0.70        45

              precision    recall  f1-score   support

         0.0       0.72      0.55      0.62       579
         1.0       0.27      0.43      0.33       222

    accuracy                           0.52       801
   macro avg       0.49      0.49      0.48       801
weighted avg       0.59      0.52      0.54       801

              preci

              precision    recall  f1-score   support

         0.0       0.45      0.95      0.61       425
         1.0       0.61      0.07      0.12       533

    accuracy                           0.46       958
   macro avg       0.53      0.51      0.36       958
weighted avg       0.54      0.46      0.34       958

              precision    recall  f1-score   support

         0.0       0.32      0.81      0.46       162
         1.0       0.78      0.28      0.41       387

    accuracy                           0.44       549
   macro avg       0.55      0.55      0.44       549
weighted avg       0.65      0.44      0.43       549

              precision    recall  f1-score   support

         0.0       0.66      0.69      0.67       267
         1.0       0.60      0.57      0.59       218

    accuracy                           0.64       485
   macro avg       0.63      0.63      0.63       485
weighted avg       0.63      0.64      0.63       485

              preci

              precision    recall  f1-score   support

         0.0       0.75      0.66      0.70      1905
         1.0       0.35      0.44      0.39       770

    accuracy                           0.60      2675
   macro avg       0.55      0.55      0.54      2675
weighted avg       0.63      0.60      0.61      2675

              precision    recall  f1-score   support

         0.0       0.41      0.51      0.46      1383
         1.0       0.55      0.45      0.50      1839

    accuracy                           0.48      3222
   macro avg       0.48      0.48      0.48      3222
weighted avg       0.49      0.48      0.48      3222

              precision    recall  f1-score   support

         0.0       0.62      0.63      0.63      1470
         1.0       0.45      0.45      0.45      1011

    accuracy                           0.56      2481
   macro avg       0.54      0.54      0.54      2481
weighted avg       0.55      0.56      0.55      2481

              preci